In [14]:
import cv2
import numpy as np
import random as rng

img = cv2.imread('IMG_2960.jpg')

In [15]:
def scale_img(img, scale_width=1000):
    scale_percent = (scale_width * 100) / img.shape[1]
    height = int(img.shape[0] * scale_percent / 100)
    dim = (scale_width, height)
    return cv2.resize(img, dim)

In [16]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
blur = cv2.blur(gray, (3, 3)) # blur the image
ret, thresh = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY)

In [17]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [6]:
hierarchy

array([[[  -1,   -1,    1,   -1],
        [   2,   -1,   -1,    0],
        [   3,    1,   -1,    0],
        ...,
        [5872, 5870,   -1, 5222],
        [  -1, 5871,   -1, 5222],
        [  -1, 5222,   -1,    0]]], dtype=int32)

In [ ]:
# create hull array for convex hull points
hull = []

drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
height, width, channels = img.shape

smalest_contour_area = height * width * 0.02 #contours smaler than 20% of the whole image should be removes

# calculate points for each contour
for i in range(len(contours)):
    # creating convex hull object for each contour
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    contour = contours[i]
    area = cv2.contourArea(contour)
    if area > smalest_contour_area:
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
    cv2.drawContours(drawing, [contour], 0, color)



cv2.imshow('Binary image',scale_img(drawing))
cv2.waitKey(0)


In [ ]:
    hull = cv2.convexHull(contour)
    phull = Polygon([[x, y] for (x, y) in
                     zip(hull[:, :, 0], hull[:, :, 1])])

def characterize_card_contour(card_contour,
                              max_segment_area,
                              image_area):
    """
    Calculates a bounding polygon for a contour, in addition
    to several charasteristic parameters.
    """
    phull = convex_hull_polygon(card_contour)
    if (phull.area < 0.1 * max_segment_area or
            phull.area < image_area / 1000.):
        # break after card size range has been explored
        continue_segmentation = False
        is_card_candidate = False
        bounding_poly = None
        crop_factor = 1.
    else:
        continue_segmentation = True
        bounding_poly = get_bounding_quad(phull)
        qc_diff = quad_corner_diff(phull, bounding_poly)
        crop_factor = min(1., (1. - qc_diff * 22. / 100.))
        is_card_candidate = bool(
            0.1 * max_segment_area < bounding_poly.area <
            image_area * 0.99 and
            qc_diff < 0.35 and
            0.25 < polygon_form_factor(bounding_poly) < 0.33)

    return (continue_segmentation,
            is_card_candidate,
            bounding_poly,
            crop_factor)

In [19]:
# create an empty black image
drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)

# draw contours and hull points
for i in range(len(contours)):
    # color_contours = (0, 255, 0) # green - color for contours
    color = (255, 0, 0) # blue - color for convex hull
    # draw ith contour
    # cv2.drawContours(drawing, contours, i, color_contours, 1, 8, hierarchy)
    # draw ith convex hull object
    cv2.drawContours(drawing, hull, i, color, 1, 8)

In [ ]:
cv2.imshow('Binary image', scale_img(drawing))
cv2.waitKey(0)